In [42]:
import math
import bz2
import json
from dateutil.parser import parse as dp
from tqdm import tqdm_notebook

In [39]:
def rfix(r):
    for f in ['APOGEE', 'ARG_OF_PERICENTER', 'BSTAR', 'ECCENTRICITY', 'INCLINATION', 'MEAN_ANOMALY', 'MEAN_MOTION','MEAN_MOTION_DDOT','MEAN_MOTION_DOT','PERIGEE','PERIOD','RA_OF_ASC_NODE','SEMIMAJOR_AXIS']:
        r[f] = float(r[f])
    for f in ['ELEMENT_SET_NO', 'NORAD_CAT_ID', 'REV_AT_EPOCH', 'OBJECT_NUMBER', 'EPHEMERIS_TYPE']:
        r[f] = int(r[f])
    
    r.pop('COMMENT', None)
    r['EPOCH'] = dp(str(r['EPOCH']))
    try:
        r['EPOCH'] = r['EPOCH'].replace(microsecond=int(r['EPOCH_MICROSECONDS']))
        r.pop('EPOCH_MICROSECONDS', None)
    except KeyError:
        pass
    return r

In [46]:
catalog = dict()
with bz2.BZ2File('/home/ckuethe/norad/daily-tle/spacetrack/spacetrack_20190924071701.json.bz2') as fd:
    tmp = json.load(fd)
    for x in tqdm_notebook(tmp):
        try:
            rfix(x)
            catalog[x['OBJECT_NUMBER']] = x
        except TypeError:
            pass

In [54]:
sorted(filtered_catalog.keys())[-1]

44533

In [50]:
filtered_catalog = dict()
for i in catalog:
    if catalog[i]['OBJECT_TYPE'] == 'PAYLOAD':
        filtered_catalog[i] = catalog[i]

In [49]:
from sgp4.io import twoline2rv
from sgp4.earth_gravity import wgs72, wgs84

In [55]:
rec = filtered_catalog[44533]

In [65]:
t = (rec['OBJECT_NAME'], rec['TLE_LINE1'], rec['TLE_LINE2'])
sgp4sat = twoline2rv(t[1], t[2], wgs72)

In [71]:
from skyfield.api import Loader, EarthSatellite, Topos

load = Loader('/home/ckuethe/.skyfield/')

In [83]:
ts = load.timescale()

now = ts.now()
sj = Topos(latitude_degrees=37.293653713,longitude_degrees=-121.918863765,elevation_m=54.341)

satsky = EarthSatellite(t[1], t[2])

lookvector = (satsky - sj)
topocentric = lookvector.at(now)

In [87]:
alt, az, distance = topocentric.altaz()

if alt.degrees > 0:
    print('The ISS is above the horizon')

print(float(alt.degrees))
print(float(az.degrees))
print(distance.km)

-83.366974773
309.325360317
13154.998898517668


In [86]:
alt.degrees

-83.36697477298688

In [ ]:
satsky.